In [1]:
import heapq
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import pandas as pd
import time
import multiprocessing
from data.worker import worker
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

## 1. 데이터 가공
userId_problemId.csv 파일 필요

In [2]:
df = pd.read_csv("./data/userId_problemId.csv").loc[:,['userId', 'problemId']]
df.head()

,userId,problemId
0,sos0911,1000
1,sos0911,1001
2,sos0911,1002
3,sos0911,1003
4,sos0911,1005


### 1) userId remapping

In [3]:
def remap_user():
    user_problem_df = pd.read_csv("./data/userId_problemId.csv").loc[:, ['userId', 'problemId']]
    user_df = pd.DataFrame({"user_id": [], "remap_id": []})
    unique_sorted_user = sorted(user_problem_df['userId'].unique().tolist())
    for i in range(len(unique_sorted_user)):
        new_user_df = pd.DataFrame({"user_id": [unique_sorted_user[i]], "remap_id": [i]})
        user_df = pd.concat((user_df, new_user_df))
        
    user_df.to_csv("./data/user_list.csv", index=False)
    print('user_list file saved successfully')

### 2) problemId remapping

In [4]:
def remap_problem():
    user_problem_df = pd.read_csv("./data/userId_problemId.csv").loc[:, ['userId', 'problemId']]
    problem_df = pd.DataFrame({"problem_id": [], "remap_id": []})
    unique_sorted_problem = sorted(user_problem_df['problemId'].unique().tolist())
    for i in range(len(unique_sorted_problem)):
        new_problem_df = pd.DataFrame({"problem_id": [unique_sorted_problem[i]], "remap_id": [i]})
        problem_df = pd.concat((problem_df, new_problem_df))
    
    problem_df.to_csv("./data/problem_list.csv", index=False)
    print('problem_list file saved successfully')

### 3) Convert userId, problemId

In [5]:
def remap_user_problem():
    user_problem_df = pd.read_csv("./data/userId_problemId.csv").loc[:, ['userId', 'problemId']]
    user_df = pd.read_csv("./data/user_list.csv")
    problem_df = pd.read_csv("./data/problem_list.csv")

    manager = multiprocessing.Manager()
    user_problem_remap_df = pd.DataFrame({'userId': [], 'problemId': []})

    for i in range(0, user_problem_df['userId'].size, 10000):
        clear_output(wait=True)
        print('Loading: [{}]'.format('-' * (i // 10000) + '>' + '-' * (83 - i // 10000)))
        
        return_dict = manager.dict()
        jobs = []
        
        for j in range(4):
            p = multiprocessing.Process(target=worker, args=(j, i + 2500 * j, user_df, problem_df, user_problem_df[i + 2500*j:i+2500*(j+1)], return_dict))
            jobs.append(p)
            p.start()
        for proc in jobs:
            proc.join()
            proc.close()
        for j in range(len(return_dict.keys())):
            pd.concat((user_problem_remap_df, return_dict[j]))
    
    user_problem_df.to_csv("./data/userId_problemId_remap.csv", index=False)
    print('userId_problemId_remap file saved successfully')

In [6]:
remap_user()

user_list file saved successfully


In [7]:
remap_problem()

problem_list file saved successfully


multiprocessing을 사용하여 30분 이상 걸리던 작업을 13분으로 단축

In [9]:
remap_user_problem()

Loading: [----------------------------------------------------------------------------------->]
userId_problemId_remap file saved successfully
790.1658284664154
